In [1]:
!pip install -q 'vanna[chromadb,openai,postgres]'
!pip install -q pandas sqlalchemy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 17.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.2/213.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import pandas as pd
from vanna.openai import OpenAI_Chat
from vanna.chromadb import ChromaDB_VectorStore
import sqlite3
import warnings
warnings.filterwarnings('ignore')  # Suppress warnings

In [3]:
class SimpleVanna(ChromaDB_VectorStore, OpenAI_Chat):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        OpenAI_Chat.__init__(self, config=config)


In [4]:
def setup_simple_vanna():
    """Simple Vanna setup with minimal configuration"""

    # IMPORTANT: Replace with your actual OpenAI API key
    api_key = "sk-"
    vn = SimpleVanna(config={
        'api_key': api_key,
        'model': 'gpt-4o-mini',
        'path': './simple_chroma'
    })

    return vn


In [5]:
def create_demo_database():
    """Create a simple demo database"""

    conn = sqlite3.connect('demo.db')

    # Create and populate customers table
    conn.execute('''
        CREATE TABLE IF NOT EXISTS customers (
            id INTEGER PRIMARY KEY,
            name TEXT,
            email TEXT,
            city TEXT
        )
    ''')

    customers = [
        (1, 'John Doe', 'john@email.com', 'New York'),
        (2, 'Jane Smith', 'jane@email.com', 'Los Angeles'),
        (3, 'Bob Johnson', 'bob@email.com', 'Chicago'),
        (4, 'Alice Brown', 'alice@email.com', 'Houston'),
        (5, 'Charlie Wilson', 'charlie@email.com', 'Phoenix')
    ]

    conn.executemany('INSERT OR REPLACE INTO customers VALUES (?, ?, ?, ?)', customers)

    # Create and populate orders table
    conn.execute('''
        CREATE TABLE IF NOT EXISTS orders (
            id INTEGER PRIMARY KEY,
            customer_id INTEGER,
            product TEXT,
            amount REAL,
            order_date TEXT,
            FOREIGN KEY (customer_id) REFERENCES customers(id)
        )
    ''')

    orders = [
        (1, 1, 'Laptop', 1200.00, '2024-01-15'),
        (2, 2, 'Phone', 800.00, '2024-01-16'),
        (3, 1, 'Tablet', 400.00, '2024-01-17'),
        (4, 3, 'Laptop', 1200.00, '2024-01-18'),
        (5, 2, 'Headphones', 150.00, '2024-01-19'),
        (6, 4, 'Phone', 800.00, '2024-01-20')
    ]

    conn.executemany('INSERT OR REPLACE INTO orders VALUES (?, ?, ?, ?, ?)', orders)
    conn.commit()
    conn.close()

    print("✅ Demo database created successfully!")

In [6]:
def train_on_schema(vn):
    """Train Vanna on database schema"""

    # Connect to database
    vn.connect_to_sqlite('demo.db')

    # Train on table schemas
    schemas = [
        """CREATE TABLE customers (
            id INTEGER PRIMARY KEY,
            name TEXT,
            email TEXT,
            city TEXT
        );""",

        """CREATE TABLE orders (
            id INTEGER PRIMARY KEY,
            customer_id INTEGER,
            product TEXT,
            amount REAL,
            order_date TEXT,
            FOREIGN KEY (customer_id) REFERENCES customers(id)
        );"""
    ]

    for schema in schemas:
        vn.train(ddl=schema)

    print("✅ Schema training completed!")

In [7]:
def train_on_examples(vn):
    """Train on question-SQL examples"""

    examples = [
        ("How many customers do we have?", "SELECT COUNT(*) FROM customers;"),
        ("What are the total sales?", "SELECT SUM(amount) FROM orders;"),
        ("Show customers from New York", "SELECT * FROM customers WHERE city = 'New York';"),
        ("What is the average order amount?", "SELECT AVG(amount) FROM orders;"),
        ("List all orders with customer names",
         "SELECT o.id, c.name, o.product, o.amount FROM orders o JOIN customers c ON o.customer_id = c.id;")
    ]

    for question, sql in examples:
        vn.train(question=question, sql=sql)

    print("✅ Example training completed!")

In [8]:
def query_database(vn, question):
    """Query the database with natural language"""

    print(f"\n🤔 Question: {question}")
    print("=" * 60)

    try:
        # Generate SQL
        sql = vn.generate_sql(question)
        print(f"🔍 Generated SQL:\n{sql}")

        # Execute SQL
        result = vn.run_sql(sql)
        print(f"\n📊 Results:")
        print(result.to_string(index=False) if not result.empty else "No results found")

        return sql, result

    except Exception as e:
        print(f"❌ Error: {e}")
        return None, None

In [9]:
def main():
    """Main function to run the demo"""

    print("🚀 Starting Natural Language to SQL Demo")
    print("=" * 50)

    # Step 1: Create demo database
    create_demo_database()

    # Step 2: Setup Vanna
    print("\n⚙️  Setting up Vanna...")
    vn = setup_simple_vanna()

    # Step 3: Train on schema
    print("\n📚 Training on database schema...")
    train_on_schema(vn)

    # Step 4: Train on examples
    print("\n📝 Training on example questions...")
    train_on_examples(vn)

    # Step 5: Test queries
    print("\n🧪 Testing natural language queries...")
    test_questions = [
        #"How many customers are there?",
        #"What is the total revenue?",
        #"Show me customers from Chicago",
        #"Which customer spent the most?",
        #"What products do we sell?",
        #"Show orders from January 2024"
    ]

    for question in test_questions:
        query_database(vn, question)
        print("\n" + "-" * 60)

    # Step 6: Interactive mode
    print("\n🎯 Interactive Mode - Ask your own questions!")
    print("Type 'quit' to exit")

    while True:
        try:
            user_question = input("\n💬 Your question: ").strip()
            if user_question.lower() in ['quit', 'exit', 'q']:
                break
            if user_question:
                query_database(vn, user_question)
        except KeyboardInterrupt:
            break

    print("\n👋 Thanks for using the Natural Language to SQL demo!")

# Additional utility functions

def add_training_data(vn, question, sql):
    """Add a single training example"""
    vn.train(question=question, sql=sql)
    print(f"✅ Added training: {question}")

def test_sql_directly(vn, sql):
    """Test SQL query directly"""
    try:
        result = vn.run_sql(sql)
        print(result)
        return result
    except Exception as e:
        print(f"Error: {e}")
        return None

def show_training_data(vn):
    """Show what Vanna has learned (if available)"""
    try:
        # This might not work with all Vanna versions
        print("Training data summary available in ChromaDB storage")
    except:
        print("Cannot display training data")

if __name__ == "__main__":
    # Run the main demo
    main()

🚀 Starting Natural Language to SQL Demo
✅ Demo database created successfully!

⚙️  Setting up Vanna...


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given



📚 Training on database schema...
Adding ddl: CREATE TABLE customers (
            id INTEGER PRIMARY KEY,
            name TEXT,
            email TEXT,
            city TEXT
        );


/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:01<00:00, 55.6MiB/s]
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionAddEvent: capture() takes 1 positional argument but 3 were given


Adding ddl: CREATE TABLE orders (
            id INTEGER PRIMARY KEY,
            customer_id INTEGER,
            product TEXT,
            amount REAL,
            order_date TEXT,
            FOREIGN KEY (customer_id) REFERENCES customers(id)
        );
✅ Schema training completed!

📝 Training on example questions...


...

SQL Prompt: [{'role': 'system', 'content': "You are a SQLite expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \nCREATE TABLE customers (\n            id INTEGER PRIMARY KEY,\n            name TEXT,\n            email TEXT,\n            city TEXT\n        );\n\nCREATE TABLE orders (\n            id INTEGER PRIMARY KEY,\n            customer_id INTEGER,\n            product TEXT,\n            amount REAL,\n            order_date TEXT,\n            FOREIGN KEY (customer_id) REFERENCES customers(id)\n        );\n\n===Response Guidelines \n1. If the provided context is sufficient, please generate a valid SQL query without any explanations for the question. \n2. If the provided context is almost sufficient but requires knowledge of a specific string in a particular column, please generate an intermediate SQL query to find the distinct strings in t


🤔 Question: Show me orders from 2024


SQL Prompt: [{'role': 'system', 'content': "You are a SQLite expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \nCREATE TABLE orders (\n            id INTEGER PRIMARY KEY,\n            customer_id INTEGER,\n            product TEXT,\n            amount REAL,\n            order_date TEXT,\n            FOREIGN KEY (customer_id) REFERENCES customers(id)\n        );\n\nCREATE TABLE customers (\n            id INTEGER PRIMARY KEY,\n            name TEXT,\n            email TEXT,\n            city TEXT\n        );\n\n===Response Guidelines \n1. If the provided context is sufficient, please generate a valid SQL query without any explanations for the question. \n2. If the provided context is almost sufficient but requires knowledge of a specific string in a particular column, please generate an intermediate SQL query to find the distinct strings in t


🤔 Question: What is the total revenue?


SQL Prompt: [{'role': 'system', 'content': "You are a SQLite expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \nCREATE TABLE orders (\n            id INTEGER PRIMARY KEY,\n            customer_id INTEGER,\n            product TEXT,\n            amount REAL,\n            order_date TEXT,\n            FOREIGN KEY (customer_id) REFERENCES customers(id)\n        );\n\nCREATE TABLE customers (\n            id INTEGER PRIMARY KEY,\n            name TEXT,\n            email TEXT,\n            city TEXT\n        );\n\n===Response Guidelines \n1. If the provided context is sufficient, please generate a valid SQL query without any explanations for the question. \n2. If the provided context is almost sufficient but requires knowledge of a specific string in a particular column, please generate an intermediate SQL query to find the distinct strings in t

# Complicated Schema

In [10]:
import os
import pandas as pd
import sqlite3
import warnings
import random
from datetime import datetime, timedelta
warnings.filterwarnings('ignore')
from vanna.openai import OpenAI_Chat
from vanna.chromadb import ChromaDB_VectorStore

In [11]:
class ComplexVanna(ChromaDB_VectorStore, OpenAI_Chat):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        OpenAI_Chat.__init__(self, config=config)

    def generate_explanation(self, sql):
        """Generate explanation for SQL query"""
        try:
            prompt = f"Explain this SQL query in simple business terms: {sql}"
            return self.submit_prompt(prompt)
        except:
            return "SQL query executed successfully - shows requested business data."


In [12]:
def setup_complex_vanna():
    """Setup Vanna with complex database configuration"""

    # IMPORTANT: Replace with your actual OpenAI API key
    api_key = "sk-"

    vn = ComplexVanna(config={
        'api_key': api_key,
        'model': 'gpt-4o-mini',  # Use gpt-4 for even better results
        'path': './complex_chroma'
    })

    # Enable LLM to see database data for better SQL generation
    vn.allow_llm_to_see_data = True

    return vn


In [13]:
def create_complex_ecommerce_database():
    """Create a complex e-commerce database with multiple related tables"""

    conn = sqlite3.connect('ecommerce_complex.db')
    cursor = conn.cursor()

    # 1. CUSTOMERS TABLE
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS customers (
            customer_id INTEGER PRIMARY KEY,
            first_name TEXT NOT NULL,
            last_name TEXT NOT NULL,
            email TEXT UNIQUE NOT NULL,
            phone TEXT,
            date_of_birth DATE,
            registration_date DATETIME,
            customer_status TEXT DEFAULT 'active', -- active, inactive, suspended
            address_id INTEGER,
            FOREIGN KEY (address_id) REFERENCES addresses(address_id)
        )
    ''')

    # 2. ADDRESSES TABLE
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS addresses (
            address_id INTEGER PRIMARY KEY,
            street_address TEXT NOT NULL,
            city TEXT NOT NULL,
            state TEXT NOT NULL,
            zip_code TEXT NOT NULL,
            country TEXT DEFAULT 'USA'
        )
    ''')

    # 3. CATEGORIES TABLE
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS categories (
            category_id INTEGER PRIMARY KEY,
            category_name TEXT NOT NULL,
            parent_category_id INTEGER,
            description TEXT,
            FOREIGN KEY (parent_category_id) REFERENCES categories(category_id)
        )
    ''')

    # 4. SUPPLIERS TABLE
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS suppliers (
            supplier_id INTEGER PRIMARY KEY,
            supplier_name TEXT NOT NULL,
            contact_person TEXT,
            email TEXT,
            phone TEXT,
            address_id INTEGER,
            rating REAL DEFAULT 0.0,
            FOREIGN KEY (address_id) REFERENCES addresses(address_id)
        )
    ''')

    # 5. PRODUCTS TABLE
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS products (
            product_id INTEGER PRIMARY KEY,
            product_name TEXT NOT NULL,
            description TEXT,
            category_id INTEGER,
            supplier_id INTEGER,
            unit_price REAL NOT NULL,
            units_in_stock INTEGER DEFAULT 0,
            reorder_level INTEGER DEFAULT 10,
            discontinued BOOLEAN DEFAULT 0,
            created_date DATETIME,
            FOREIGN KEY (category_id) REFERENCES categories(category_id),
            FOREIGN KEY (supplier_id) REFERENCES suppliers(supplier_id)
        )
    ''')

    # 6. ORDERS TABLE
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS orders (
            order_id INTEGER PRIMARY KEY,
            customer_id INTEGER NOT NULL,
            order_date DATETIME NOT NULL,
            required_date DATE,
            shipped_date DATE,
            ship_address_id INTEGER,
            freight REAL DEFAULT 0.0,
            order_status TEXT DEFAULT 'pending', -- pending, processing, shipped, delivered, cancelled
            payment_method TEXT, -- credit_card, debit_card, paypal, bank_transfer
            total_amount REAL,
            FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
            FOREIGN KEY (ship_address_id) REFERENCES addresses(address_id)
        )
    ''')

    # 7. ORDER_DETAILS TABLE (Many-to-Many between Orders and Products)
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS order_details (
            order_detail_id INTEGER PRIMARY KEY,
            order_id INTEGER NOT NULL,
            product_id INTEGER NOT NULL,
            unit_price REAL NOT NULL,
            quantity INTEGER NOT NULL,
            discount REAL DEFAULT 0.0,
            FOREIGN KEY (order_id) REFERENCES orders(order_id),
            FOREIGN KEY (product_id) REFERENCES products(product_id)
        )
    ''')

    # 8. REVIEWS TABLE
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS reviews (
            review_id INTEGER PRIMARY KEY,
            product_id INTEGER NOT NULL,
            customer_id INTEGER NOT NULL,
            rating INTEGER CHECK (rating >= 1 AND rating <= 5),
            review_text TEXT,
            review_date DATETIME,
            helpful_votes INTEGER DEFAULT 0,
            FOREIGN KEY (product_id) REFERENCES products(product_id),
            FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
        )
    ''')

    # 9. INVENTORY_TRANSACTIONS TABLE
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS inventory_transactions (
            transaction_id INTEGER PRIMARY KEY,
            product_id INTEGER NOT NULL,
            transaction_type TEXT NOT NULL, -- 'purchase', 'sale', 'adjustment', 'return'
            quantity_change INTEGER NOT NULL,
            transaction_date DATETIME,
            reference_id INTEGER, -- Could reference order_id or supplier invoice
            notes TEXT,
            FOREIGN KEY (product_id) REFERENCES products(product_id)
        )
    ''')

    # 10. EMPLOYEE TABLE (for sales tracking)
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS employees (
            employee_id INTEGER PRIMARY KEY,
            first_name TEXT NOT NULL,
            last_name TEXT NOT NULL,
            title TEXT,
            department TEXT,
            hire_date DATE,
            salary REAL,
            manager_id INTEGER,
            FOREIGN KEY (manager_id) REFERENCES employees(employee_id)
        )
    ''')

    conn.commit()
    print("✅ Complex database schema created successfully!")
    return conn


In [14]:
def populate_complex_database():
    """Populate the database with realistic sample data"""

    conn = sqlite3.connect('ecommerce_complex.db')
    cursor = conn.cursor()

    # Clear existing data
    tables = ['inventory_transactions', 'reviews', 'order_details', 'orders',
              'products', 'customers', 'suppliers', 'categories', 'addresses', 'employees']
    for table in tables:
        cursor.execute(f'DELETE FROM {table}')

    # 1. ADDRESSES
    addresses = [
        (1, '123 Main St', 'New York', 'NY', '10001', 'USA'),
        (2, '456 Oak Ave', 'Los Angeles', 'CA', '90210', 'USA'),
        (3, '789 Pine Rd', 'Chicago', 'IL', '60601', 'USA'),
        (4, '321 Elm St', 'Houston', 'TX', '77001', 'USA'),
        (5, '654 Maple Dr', 'Phoenix', 'AZ', '85001', 'USA'),
        (6, '987 Cedar Ln', 'Philadelphia', 'PA', '19101', 'USA'),
        (7, '147 Birch St', 'San Antonio', 'TX', '78201', 'USA'),
        (8, '258 Spruce Ave', 'San Diego', 'CA', '92101', 'USA'),
        (9, '369 Willow Rd', 'Dallas', 'TX', '75201', 'USA'),
        (10, '741 Ash St', 'San Jose', 'CA', '95101', 'USA')
    ]
    cursor.executemany('INSERT INTO addresses VALUES (?, ?, ?, ?, ?, ?)', addresses)

    # 2. CATEGORIES (with hierarchical structure)
    categories = [
        (1, 'Electronics', None, 'All electronic devices and accessories'),
        (2, 'Computers', 1, 'Desktop and laptop computers'),
        (3, 'Mobile Devices', 1, 'Smartphones and tablets'),
        (4, 'Audio', 1, 'Headphones, speakers, and audio equipment'),
        (5, 'Home & Garden', None, 'Home improvement and garden supplies'),
        (6, 'Furniture', 5, 'Indoor and outdoor furniture'),
        (7, 'Tools', 5, 'Hand tools and power tools'),
        (8, 'Clothing', None, 'Apparel and fashion accessories'),
        (9, 'Sports', None, 'Sports and outdoor recreation equipment'),
        (10, 'Books', None, 'Physical and digital books')
    ]
    cursor.executemany('INSERT INTO categories VALUES (?, ?, ?, ?)', categories)

    # 3. SUPPLIERS
    suppliers = [
        (1, 'TechCorp Inc', 'John Smith', 'john@techcorp.com', '555-0101', 1, 4.5),
        (2, 'Global Electronics', 'Sarah Johnson', 'sarah@globalelec.com', '555-0102', 2, 4.2),
        (3, 'Home Solutions LLC', 'Mike Brown', 'mike@homesolutions.com', '555-0103', 3, 4.8),
        (4, 'Fashion Forward', 'Lisa Davis', 'lisa@fashionforward.com', '555-0104', 4, 4.0),
        (5, 'Sports World', 'Tom Wilson', 'tom@sportsworld.com', '555-0105', 5, 4.6)
    ]
    cursor.executemany('INSERT INTO suppliers VALUES (?, ?, ?, ?, ?, ?, ?)', suppliers)

    # 4. PRODUCTS
    products = [
        (1, 'MacBook Pro 16"', 'High-performance laptop for professionals', 2, 1, 2499.99, 25, 5, 0, '2024-01-01'),
        (2, 'iPhone 15 Pro', 'Latest smartphone with advanced camera', 3, 1, 1199.99, 50, 10, 0, '2024-01-01'),
        (3, 'Dell XPS 13', 'Ultrabook with excellent display', 2, 2, 1299.99, 30, 5, 0, '2024-01-02'),
        (4, 'Sony WH-1000XM5', 'Noise-canceling wireless headphones', 4, 2, 399.99, 75, 15, 0, '2024-01-03'),
        (5, 'Samsung Galaxy S24', 'Android flagship smartphone', 3, 2, 999.99, 40, 10, 0, '2024-01-04'),
        (6, 'iPad Pro 12.9"', 'Professional tablet for creative work', 3, 1, 1099.99, 35, 8, 0, '2024-01-05'),
        (7, 'Gaming Chair Pro', 'Ergonomic gaming chair with RGB lighting', 6, 3, 299.99, 20, 5, 0, '2024-01-06'),
        (8, 'Wireless Mouse Elite', 'High-precision wireless gaming mouse', 2, 1, 79.99, 100, 20, 0, '2024-01-07'),
        (9, 'Bluetooth Speaker', 'Portable waterproof bluetooth speaker', 4, 2, 149.99, 60, 15, 0, '2024-01-08'),
        (10, 'Running Shoes', 'Professional running shoes for athletes', 9, 5, 129.99, 80, 20, 0, '2024-01-09'),
        (11, 'Yoga Mat Premium', 'Non-slip yoga mat with carrying strap', 9, 5, 49.99, 45, 10, 0, '2024-01-10'),
        (12, 'Coffee Maker Deluxe', 'Programmable coffee maker with timer', 5, 3, 199.99, 25, 5, 0, '2024-01-11'),
        (13, 'Laptop Stand Adjustable', 'Ergonomic aluminum laptop stand', 2, 3, 59.99, 70, 15, 0, '2024-01-12'),
        (14, 'Wireless Earbuds Pro', 'True wireless earbuds with ANC', 4, 1, 249.99, 85, 20, 0, '2024-01-13'),
        (15, 'Smart Watch Series 9', 'Fitness tracking smartwatch', 3, 1, 449.99, 55, 12, 0, '2024-01-14')
    ]
    cursor.executemany('INSERT INTO products VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', products)

    # 5. CUSTOMERS
    customers = [
        (1, 'John', 'Doe', 'john.doe@email.com', '555-1001', '1985-03-15', '2023-01-15 10:30:00', 'active', 1),
        (2, 'Jane', 'Smith', 'jane.smith@email.com', '555-1002', '1990-07-22', '2023-02-20 14:45:00', 'active', 2),
        (3, 'Bob', 'Johnson', 'bob.johnson@email.com', '555-1003', '1978-11-08', '2023-03-10 09:15:00', 'active', 3),
        (4, 'Alice', 'Brown', 'alice.brown@email.com', '555-1004', '1992-05-30', '2023-04-05 16:20:00', 'active', 4),
        (5, 'Charlie', 'Wilson', 'charlie.wilson@email.com', '555-1005', '1988-09-12', '2023-05-18 11:10:00', 'active', 5),
        (6, 'Diana', 'Davis', 'diana.davis@email.com', '555-1006', '1995-12-03', '2023-06-25 13:30:00', 'active', 6),
        (7, 'Edward', 'Miller', 'edward.miller@email.com', '555-1007', '1983-04-17', '2023-07-12 15:45:00', 'active', 7),
        (8, 'Fiona', 'Garcia', 'fiona.garcia@email.com', '555-1008', '1991-08-25', '2023-08-30 10:25:00', 'active', 8),
        (9, 'George', 'Martinez', 'george.martinez@email.com', '555-1009', '1987-01-14', '2023-09-15 12:40:00', 'active', 9),
        (10, 'Helen', 'Anderson', 'helen.anderson@email.com', '555-1010', '1994-06-09', '2023-10-20 14:15:00', 'active', 10)
    ]
    cursor.executemany('INSERT INTO customers VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)', customers)

    # 6. EMPLOYEES
    employees = [
        (1, 'Michael', 'Scott', 'Regional Manager', 'Sales', '2020-01-15', 75000, None),
        (2, 'Dwight', 'Schrute', 'Sales Representative', 'Sales', '2020-03-01', 50000, 1),
        (3, 'Jim', 'Halpert', 'Sales Representative', 'Sales', '2020-04-15', 52000, 1),
        (4, 'Pam', 'Beesly', 'Receptionist', 'Administration', '2020-02-01', 35000, 1),
        (5, 'Angela', 'Martin', 'Accountant', 'Accounting', '2020-01-20', 45000, None)
    ]
    cursor.executemany('INSERT INTO employees VALUES (?, ?, ?, ?, ?, ?, ?, ?)', employees)

    # 7. ORDERS (with realistic dates and progression)
    orders = [
        (1, 1, '2024-01-15 10:30:00', '2024-01-20', '2024-01-18', 1, 15.99, 'delivered', 'credit_card', 2579.98),
        (2, 2, '2024-01-16 14:20:00', '2024-01-21', '2024-01-19', 2, 12.50, 'delivered', 'paypal', 1449.98),
        (3, 3, '2024-01-17 09:45:00', '2024-01-22', '2024-01-20', 3, 18.75, 'delivered', 'debit_card', 479.98),
        (4, 4, '2024-01-18 16:15:00', '2024-01-23', '2024-01-21', 4, 9.99, 'delivered', 'credit_card', 409.98),
        (5, 5, '2024-01-19 11:30:00', '2024-01-24', '2024-01-22', 5, 14.25, 'delivered', 'bank_transfer', 1549.97),
        (6, 6, '2024-01-20 13:45:00', '2024-01-25', '2024-01-23', 6, 11.80, 'delivered', 'credit_card', 1359.97),
        (7, 7, '2024-01-21 15:20:00', '2024-01-26', '2024-01-24', 7, 22.50, 'delivered', 'paypal', 379.98),
        (8, 8, '2024-01-22 10:10:00', '2024-01-27', '2024-01-25', 8, 8.99, 'delivered', 'debit_card', 299.98),
        (9, 9, '2024-01-23 12:35:00', '2024-01-28', '2024-01-26', 9, 16.75, 'delivered', 'credit_card', 179.98),
        (10, 10, '2024-01-24 14:50:00', '2024-01-29', '2024-01-27', 10, 13.25, 'delivered', 'paypal', 629.97),
        (11, 1, '2024-02-01 09:15:00', '2024-02-06', '2024-02-04', 1, 19.99, 'delivered', 'credit_card', 649.97),
        (12, 3, '2024-02-05 16:40:00', '2024-02-10', '2024-02-08', 3, 25.50, 'delivered', 'bank_transfer', 1259.97),
        (13, 5, '2024-02-10 11:25:00', '2024-02-15', '2024-02-13', 5, 17.75, 'delivered', 'credit_card', 579.97),
        (14, 7, '2024-02-14 13:30:00', '2024-02-19', '2024-02-17', 7, 21.25, 'shipped', 'paypal', 899.97),
        (15, 9, '2024-02-18 15:45:00', '2024-02-23', None, 9, 14.50, 'processing', 'debit_card', 329.97)
    ]
    cursor.executemany('INSERT INTO orders VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', orders)

    # 8. ORDER_DETAILS (Multiple products per order)
    order_details = [
        # Order 1: MacBook Pro + Mouse
        (1, 1, 1, 2499.99, 1, 0.0),
        (2, 1, 8, 79.99, 1, 0.0),

        # Order 2: Dell XPS + Headphones
        (3, 2, 3, 1299.99, 1, 0.0),
        (4, 2, 4, 399.99, 1, 0.0),

        # Order 3: Audio products
        (5, 3, 4, 399.99, 1, 0.05),  # 5% discount
        (6, 3, 9, 149.99, 1, 0.0),

        # Order 4: Sony Headphones
        (7, 4, 4, 399.99, 1, 0.0),

        # Order 5: Samsung Galaxy + iPad
        (8, 5, 5, 999.99, 1, 0.0),
        (9, 5, 6, 1099.99, 1, 0.0),

        # Order 6: iPad + Laptop Stand
        (10, 6, 6, 1099.99, 1, 0.0),
        (11, 6, 13, 59.99, 1, 0.0),

        # Order 7: Gaming Chair + Mouse
        (12, 7, 7, 299.99, 1, 0.0),
        (13, 7, 8, 79.99, 1, 0.0),

        # Order 8: Sports equipment
        (14, 8, 10, 129.99, 1, 0.0),
        (15, 8, 11, 49.99, 1, 0.0),

        # Order 9: Bluetooth Speaker
        (16, 9, 9, 149.99, 1, 0.0),

        # Order 10: Multiple accessories
        (17, 10, 8, 79.99, 1, 0.0),
        (18, 10, 13, 59.99, 1, 0.0),
        (19, 10, 14, 249.99, 1, 0.0),

        # Order 11: Smart Watch + Earbuds
        (20, 11, 15, 449.99, 1, 0.0),
        (21, 11, 14, 249.99, 1, 0.0),

        # Order 12: iPhone + iPad
        (22, 12, 2, 1199.99, 1, 0.0),
        (23, 12, 6, 1099.99, 1, 0.0),

        # Order 13: Multiple small items
        (24, 13, 8, 79.99, 2, 0.1),  # 10% discount for quantity
        (25, 13, 13, 59.99, 3, 0.05),
        (26, 13, 11, 49.99, 2, 0.0),

        # Order 14: High-value order
        (27, 14, 1, 2499.99, 1, 0.0),
        (28, 14, 4, 399.99, 1, 0.0),

        # Order 15: Pending order
        (29, 15, 10, 129.99, 1, 0.0),
        (30, 15, 11, 49.99, 2, 0.0),
        (31, 15, 9, 149.99, 1, 0.0)
    ]
    cursor.executemany('INSERT INTO order_details VALUES (?, ?, ?, ?, ?, ?)', order_details)

    # 9. REVIEWS
    reviews = [
        (1, 1, 1, 5, 'Excellent laptop! Fast and reliable for development work.', '2024-01-25 10:15:00', 15),
        (2, 3, 2, 4, 'Great ultrabook, but battery could be better.', '2024-01-28 14:30:00', 8),
        (3, 4, 3, 5, 'Best noise-canceling headphones I have ever used!', '2024-01-30 16:45:00', 22),
        (4, 2, 4, 4, 'Good phone overall, camera is impressive.', '2024-02-02 09:20:00', 12),
        (5, 8, 5, 3, 'Mouse is okay but not as responsive as expected.', '2024-02-05 11:35:00', 5),
        (6, 9, 8, 5, 'Perfect speaker for outdoor activities!', '2024-02-08 13:50:00', 18),
        (7, 7, 7, 4, 'Comfortable gaming chair, RGB is nice touch.', '2024-02-12 15:25:00', 9),
        (8, 6, 6, 5, 'iPad Pro is amazing for digital art and note-taking.', '2024-02-15 12:40:00', 14),
        (9, 10, 5, 4, 'Samsung Galaxy has great features and display.', '2024-02-18 10:55:00', 7),
        (10, 15, 1, 5, 'MacBook Pro exceeded my expectations!', '2024-02-20 14:20:00', 20)
    ]
    cursor.executemany('INSERT INTO reviews VALUES (?, ?, ?, ?, ?, ?, ?)', reviews)

    # 10. INVENTORY_TRANSACTIONS
    inventory_transactions = [
        (1, 1, 'purchase', 50, '2024-01-01 08:00:00', None, 'Initial stock'),
        (2, 2, 'purchase', 100, '2024-01-01 08:00:00', None, 'Initial stock'),
        (3, 3, 'purchase', 60, '2024-01-02 08:00:00', None, 'Initial stock'),
        (4, 1, 'sale', -1, '2024-01-15 10:30:00', 1, 'Order #1'),
        (5, 8, 'sale', -1, '2024-01-15 10:30:00', 1, 'Order #1'),
        (6, 3, 'sale', -1, '2024-01-16 14:20:00', 2, 'Order #2'),
        (7, 4, 'sale', -1, '2024-01-16 14:20:00', 2, 'Order #2'),
        (8, 2, 'purchase', 25, '2024-02-01 09:00:00', None, 'Restock'),
        (9, 4, 'adjustment', -5, '2024-02-15 14:00:00', None, 'Damaged units'),
        (10, 15, 'purchase', 30, '2024-02-20 10:00:00', None, 'New product launch')
    ]
    cursor.executemany('INSERT INTO inventory_transactions VALUES (?, ?, ?, ?, ?, ?, ?)', inventory_transactions)

    conn.commit()
    conn.close()
    print("✅ Complex database populated with realistic data!")


In [15]:
def train_complex_schema(vn):
    """Train Vanna on the complex database schema"""

    vn.connect_to_sqlite('ecommerce_complex.db')

    # Database schema training
    schemas = [
        """CREATE TABLE customers (
            customer_id INTEGER PRIMARY KEY,
            first_name TEXT NOT NULL,
            last_name TEXT NOT NULL,
            email TEXT UNIQUE NOT NULL,
            phone TEXT,
            date_of_birth DATE,
            registration_date DATETIME,
            customer_status TEXT DEFAULT 'active',
            address_id INTEGER,
            FOREIGN KEY (address_id) REFERENCES addresses(address_id)
        );""",

        """CREATE TABLE addresses (
            address_id INTEGER PRIMARY KEY,
            street_address TEXT NOT NULL,
            city TEXT NOT NULL,
            state TEXT NOT NULL,
            zip_code TEXT NOT NULL,
            country TEXT DEFAULT 'USA'
        );""",

        """CREATE TABLE categories (
            category_id INTEGER PRIMARY KEY,
            category_name TEXT NOT NULL,
            parent_category_id INTEGER,
            description TEXT,
            FOREIGN KEY (parent_category_id) REFERENCES categories(category_id)
        );""",

        """CREATE TABLE suppliers (
            supplier_id INTEGER PRIMARY KEY,
            supplier_name TEXT NOT NULL,
            contact_person TEXT,
            email TEXT,
            phone TEXT,
            address_id INTEGER,
            rating REAL DEFAULT 0.0,
            FOREIGN KEY (address_id) REFERENCES addresses(address_id)
        );""",

        """CREATE TABLE products (
            product_id INTEGER PRIMARY KEY,
            product_name TEXT NOT NULL,
            description TEXT,
            category_id INTEGER,
            supplier_id INTEGER,
            unit_price REAL NOT NULL,
            units_in_stock INTEGER DEFAULT 0,
            reorder_level INTEGER DEFAULT 10,
            discontinued BOOLEAN DEFAULT 0,
            created_date DATETIME,
            FOREIGN KEY (category_id) REFERENCES categories(category_id),
            FOREIGN KEY (supplier_id) REFERENCES suppliers(supplier_id)
        );""",

        """CREATE TABLE orders (
            order_id INTEGER PRIMARY KEY,
            customer_id INTEGER NOT NULL,
            order_date DATETIME NOT NULL,
            required_date DATE,
            shipped_date DATE,
            ship_address_id INTEGER,
            freight REAL DEFAULT 0.0,
            order_status TEXT DEFAULT 'pending',
            payment_method TEXT,
            total_amount REAL,
            FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
            FOREIGN KEY (ship_address_id) REFERENCES addresses(address_id)
        );""",

        """CREATE TABLE order_details (
            order_detail_id INTEGER PRIMARY KEY,
            order_id INTEGER NOT NULL,
            product_id INTEGER NOT NULL,
            unit_price REAL NOT NULL,
            quantity INTEGER NOT NULL,
            discount REAL DEFAULT 0.0,
            FOREIGN KEY (order_id) REFERENCES orders(order_id),
            FOREIGN KEY (product_id) REFERENCES products(product_id)
        );""",

        """CREATE TABLE reviews (
            review_id INTEGER PRIMARY KEY,
            product_id INTEGER NOT NULL,
            customer_id INTEGER NOT NULL,
            rating INTEGER CHECK (rating >= 1 AND rating <= 5),
            review_text TEXT,
            review_date DATETIME,
            helpful_votes INTEGER DEFAULT 0,
            FOREIGN KEY (product_id) REFERENCES products(product_id),
            FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
        );""",

        """CREATE TABLE inventory_transactions (
            transaction_id INTEGER PRIMARY KEY,
            product_id INTEGER NOT NULL,
            transaction_type TEXT NOT NULL,
            quantity_change INTEGER NOT NULL,
            transaction_date DATETIME,
            reference_id INTEGER,
            notes TEXT,
            FOREIGN KEY (product_id) REFERENCES products(product_id)
        );""",

        """CREATE TABLE employees (
            employee_id INTEGER PRIMARY KEY,
            first_name TEXT NOT NULL,
            last_name TEXT NOT NULL,
            title TEXT,
            department TEXT,
            hire_date DATE,
            salary REAL,
            manager_id INTEGER,
            FOREIGN KEY (manager_id) REFERENCES employees(employee_id)
        );"""
    ]

    for schema in schemas:
        vn.train(ddl=schema)

    print("✅ Complex schema training completed!")

In [16]:
def train_complex_examples(vn):
    """Train on complex multi-table queries and join examples"""

    training_examples = [
        # Basic queries
        {
            "question": "How many customers do we have?",
            "sql": "SELECT COUNT(*) as customer_count FROM customers;"
        },
        {
            "question": "What is our total revenue?",
            "sql": "SELECT SUM(total_amount) as total_revenue FROM orders WHERE order_status = 'delivered';"
        },
        {
            "question": "How many products do we sell?",
            "sql": "SELECT COUNT(*) as product_count FROM products WHERE discontinued = 0;"
        },

        # Simple joins
        {
            "question": "Show all customers with their addresses",
            "sql": """SELECT c.customer_id, c.first_name, c.last_name, c.email,
                     a.street_address, a.city, a.state, a.zip_code
                     FROM customers c
                     LEFT JOIN addresses a ON c.address_id = a.address_id;"""
        },
        {
            "question": "List all products with their categories",
            "sql": """SELECT p.product_name, p.unit_price, c.category_name, p.units_in_stock
                     FROM products p
                     LEFT JOIN categories c ON p.category_id = c.category_id;"""
        },
        {
            "question": "Show products with their suppliers",
            "sql": """SELECT p.product_name, p.unit_price, s.supplier_name, s.rating
                     FROM products p
                     LEFT JOIN suppliers s ON p.supplier_id = s.supplier_id;"""
        },

        # Complex multi-table joins
        {
            "question": "Show all orders with customer details and shipping addresses",
            "sql": """SELECT o.order_id, c.first_name, c.last_name, c.email,
                     o.order_date, o.total_amount, o.order_status,
                     a.street_address, a.city, a.state
                     FROM orders o
                     JOIN customers c ON o.customer_id = c.customer_id
                     LEFT JOIN addresses a ON o.ship_address_id = a.address_id;"""
        },
        {
            "question": "List order details with product and customer information",
            "sql": """SELECT od.order_id, c.first_name, c.last_name,
                     p.product_name, od.quantity, od.unit_price,
                     (od.quantity * od.unit_price * (1 - od.discount)) as line_total
                     FROM order_details od
                     JOIN orders o ON od.order_id = o.order_id
                     JOIN customers c ON o.customer_id = c.customer_id
                     JOIN products p ON od.product_id = p.product_id;"""
        },
        {
            "question": "Show products with their categories, suppliers, and current stock",
            "sql": """SELECT p.product_name, c.category_name, s.supplier_name,
                     p.unit_price, p.units_in_stock, p.reorder_level,
                     CASE WHEN p.units_in_stock <= p.reorder_level THEN 'Low Stock' ELSE 'In Stock' END as stock_status
                     FROM products p
                     LEFT JOIN categories c ON p.category_id = c.category_id
                     LEFT JOIN suppliers s ON p.supplier_id = s.supplier_id;"""
        },

        # Aggregation with joins
        {
            "question": "What are the total sales by category?",
            "sql": """SELECT c.category_name, SUM(od.quantity * od.unit_price * (1 - od.discount)) as total_sales
                     FROM order_details od
                     JOIN products p ON od.product_id = p.product_id
                     JOIN categories c ON p.category_id = c.category_id
                     JOIN orders o ON od.order_id = o.order_id
                     WHERE o.order_status = 'delivered'
                     GROUP BY c.category_id, c.category_name
                     ORDER BY total_sales DESC;"""
        },
        {
            "question": "Which customers have spent the most money?",
            "sql": """SELECT c.first_name, c.last_name, c.email,
                     SUM(o.total_amount) as total_spent,
                     COUNT(o.order_id) as order_count
                     FROM customers c
                     JOIN orders o ON c.customer_id = o.customer_id
                     WHERE o.order_status = 'delivered'
                     GROUP BY c.customer_id, c.first_name, c.last_name, c.email
                     ORDER BY total_spent DESC
                     LIMIT 10;"""
        },
        {
            "question": "What are the best selling products by quantity?",
            "sql": """SELECT p.product_name, c.category_name,
                     SUM(od.quantity) as total_quantity_sold,
                     SUM(od.quantity * od.unit_price * (1 - od.discount)) as total_revenue
                     FROM order_details od
                     JOIN products p ON od.product_id = p.product_id
                     LEFT JOIN categories c ON p.category_id = c.category_id
                     JOIN orders o ON od.order_id = o.order_id
                     WHERE o.order_status = 'delivered'
                     GROUP BY p.product_id, p.product_name, c.category_name
                     ORDER BY total_quantity_sold DESC
                     LIMIT 10;"""
        },

        # Geographic analysis
        {
            "question": "Show sales by state",
            "sql": """SELECT a.state, COUNT(DISTINCT o.order_id) as order_count,
                     SUM(o.total_amount) as total_sales
                     FROM orders o
                     JOIN addresses a ON o.ship_address_id = a.address_id
                     WHERE o.order_status = 'delivered'
                     GROUP BY a.state
                     ORDER BY total_sales DESC;"""
        },
        {
            "question": "Which cities have the most customers?",
            "sql": """SELECT a.city, a.state, COUNT(c.customer_id) as customer_count
                     FROM customers c
                     JOIN addresses a ON c.address_id = a.address_id
                     GROUP BY a.city, a.state
                     ORDER BY customer_count DESC;"""
        },

        # Time-based analysis
        {
            "question": "Show monthly sales for 2024",
            "sql": """SELECT strftime('%Y-%m', o.order_date) as month,
                     COUNT(o.order_id) as order_count,
                     SUM(o.total_amount) as monthly_sales
                     FROM orders o
                     WHERE o.order_status = 'delivered'
                     AND strftime('%Y', o.order_date) = '2024'
                     GROUP BY strftime('%Y-%m', o.order_date)
                     ORDER BY month;"""
        },
        {
            "question": "Show average order processing time",
            "sql": """SELECT AVG(julianday(shipped_date) - julianday(order_date)) as avg_processing_days
                     FROM orders
                     WHERE shipped_date IS NOT NULL AND order_status = 'delivered';"""
        },

        # Product reviews and ratings
        {
            "question": "Show products with their average ratings",
            "sql": """SELECT p.product_name, c.category_name,
                     AVG(r.rating) as avg_rating,
                     COUNT(r.review_id) as review_count,
                     SUM(r.helpful_votes) as total_helpful_votes
                     FROM products p
                     LEFT JOIN reviews r ON p.product_id = r.product_id
                     LEFT JOIN categories c ON p.category_id = c.category_id
                     GROUP BY p.product_id, p.product_name, c.category_name
                     HAVING COUNT(r.review_id) > 0
                     ORDER BY avg_rating DESC, review_count DESC;"""
        },
        {
            "question": "Which customers write the most reviews?",
            "sql": """SELECT c.first_name, c.last_name, c.email,
                     COUNT(r.review_id) as review_count,
                     AVG(r.rating) as avg_rating_given
                     FROM customers c
                     JOIN reviews r ON c.customer_id = r.customer_id
                     GROUP BY c.customer_id, c.first_name, c.last_name, c.email
                     ORDER BY review_count DESC;"""
        },

        # Inventory analysis
        {
            "question": "Show products that need restocking",
            "sql": """SELECT p.product_name, c.category_name, s.supplier_name,
                     p.units_in_stock, p.reorder_level,
                     (p.reorder_level - p.units_in_stock) as units_needed
                     FROM products p
                     LEFT JOIN categories c ON p.category_id = c.category_id
                     LEFT JOIN suppliers s ON p.supplier_id = s.supplier_id
                     WHERE p.units_in_stock <= p.reorder_level
                     AND p.discontinued = 0
                     ORDER BY units_needed DESC;"""
        },
        {
            "question": "Show inventory movement history for a specific product",
            "sql": """SELECT p.product_name, it.transaction_type, it.quantity_change,
                     it.transaction_date, it.notes,
                     SUM(it.quantity_change) OVER (ORDER BY it.transaction_date) as running_total
                     FROM inventory_transactions it
                     JOIN products p ON it.product_id = p.product_id
                     WHERE p.product_name = 'MacBook Pro 16"'
                     ORDER BY it.transaction_date;"""
        },

        # Supplier analysis
        {
            "question": "Show supplier performance with sales data",
            "sql": """SELECT s.supplier_name, s.rating,
                     COUNT(DISTINCT p.product_id) as products_supplied,
                     SUM(od.quantity * od.unit_price * (1 - od.discount)) as total_sales_generated
                     FROM suppliers s
                     LEFT JOIN products p ON s.supplier_id = p.supplier_id
                     LEFT JOIN order_details od ON p.product_id = od.product_id
                     LEFT JOIN orders o ON od.order_id = o.order_id
                     WHERE o.order_status = 'delivered' OR o.order_status IS NULL
                     GROUP BY s.supplier_id, s.supplier_name, s.rating
                     ORDER BY total_sales_generated DESC;"""
        },

        # Customer behavior analysis
        {
            "question": "Show customer purchase patterns by category",
            "sql": """SELECT c.first_name, c.last_name, cat.category_name,
                     COUNT(od.order_detail_id) as items_purchased,
                     SUM(od.quantity * od.unit_price * (1 - od.discount)) as spent_in_category
                     FROM customers c
                     JOIN orders o ON c.customer_id = o.customer_id
                     JOIN order_details od ON o.order_id = od.order_id
                     JOIN products p ON od.product_id = p.product_id
                     JOIN categories cat ON p.category_id = cat.category_id
                     WHERE o.order_status = 'delivered'
                     GROUP BY c.customer_id, c.first_name, c.last_name, cat.category_id, cat.category_name
                     ORDER BY c.last_name, spent_in_category DESC;"""
        },

        # Advanced analytics
        {
            "question": "Show customer lifetime value analysis",
            "sql": """SELECT c.customer_id, c.first_name, c.last_name,
                     MIN(o.order_date) as first_order_date,
                     MAX(o.order_date) as last_order_date,
                     COUNT(o.order_id) as total_orders,
                     SUM(o.total_amount) as lifetime_value,
                     AVG(o.total_amount) as avg_order_value,
                     julianday('now') - julianday(MAX(o.order_date)) as days_since_last_order
                     FROM customers c
                     JOIN orders o ON c.customer_id = o.customer_id
                     WHERE o.order_status = 'delivered'
                     GROUP BY c.customer_id, c.first_name, c.last_name
                     ORDER BY lifetime_value DESC;"""
        }
    ]

    for example in training_examples:
        vn.train(question=example["question"], sql=example["sql"])

    print("✅ Complex query training completed!")

In [17]:
def query_complex_database(vn, question):
    """Query the complex database with enhanced error handling and formatting"""

    print(f"\n🤔 Question: {question}")
    print("=" * 80)

    try:
        # Generate SQL
        sql = vn.generate_sql(question)
        print(f"🔍 Generated SQL:")
        print(f"```sql\n{sql}\n```")

        # Execute SQL
        result = vn.run_sql(sql)

        # Format and display results
        print(f"\n📊 Results:")
        if result.empty:
            print("No results found")
        else:
            # Limit display for large result sets
            if len(result) > 20:
                print(f"Showing first 20 rows of {len(result)} total results:")
                print(result.head(20).to_string(index=False))
                print(f"... and {len(result) - 20} more rows")
            else:
                print(result.to_string(index=False))

        # Generate explanation
        try:
            explanation = vn.generate_explanation(sql)
            print(f"\n💡 Explanation: {explanation}")
        except:
            print(f"\n💡 This query analyzes {question.lower()} using data from multiple related tables.")

        return sql, result

    except Exception as e:
        print(f"❌ Error: {e}")
        print("💡 Try rephrasing your question or being more specific.")
        return None, None

In [18]:
def main_complex():
    """Main function for complex database demo"""

    print("🚀 Starting Complex Multi-Table Natural Language to SQL Demo")
    print("=" * 80)

    # Step 1: Create and populate complex database
    print("\n📦 Creating complex e-commerce database...")
    create_complex_ecommerce_database()
    populate_complex_database()

    # Step 2: Setup Vanna
    print("\n⚙️  Setting up Vanna with advanced configuration...")
    vn = setup_complex_vanna()

    # Step 3: Train on complex schema
    print("\n📚 Training on complex database schema...")
    train_complex_schema(vn)

    # Step 4: Train on complex examples
    print("\n📝 Training on advanced query examples...")
    train_complex_examples(vn)

    # Step 5: Test complex queries
    print("\n🧪 Testing complex multi-table queries...")

    complex_test_questions = [
        # Basic analytics
        "How many customers do we have and what is our total revenue?",
        "Show me the top 5 best selling products",
        "Which customers have spent more than $1000?",

        # Geographic analysis
        "What are the sales by state?",
        "Which cities have the most customers?",

        # Product analysis
        "Show products with their average ratings and review counts",
        "Which products need restocking?",
        "What are the total sales by category?",

        # Customer analysis
        "Who are our most valuable customers?",
        "Show customer purchase patterns by category",

        # Time-based analysis
        "Show monthly sales trends for 2024",
        "What is the average order processing time?",

        # Supplier analysis
        "How are our suppliers performing?",
        "Which supplier generates the most revenue?",

        # Complex joins
        "Show all orders with complete customer and product details",
        "List customers with their addresses and total purchases",

        # Advanced analytics
        "Calculate customer lifetime value",
        "Show inventory movement history",
        "Analyze review sentiment by product category"
    ]

    for question in complex_test_questions:
        query_complex_database(vn, question)
        print("\n" + "─" * 80)

    # Step 6: Interactive mode with suggestions
    print("\n🎯 Interactive Mode - Ask Complex Questions!")
    print("💡 Try asking about:")
    print("   • Customer analytics: 'Who are repeat customers?'")
    print("   • Product performance: 'Which electronics sell best?'")
    print("   • Geographic trends: 'Sales by California cities'")
    print("   • Time analysis: 'Weekly sales patterns'")
    print("   • Inventory: 'Products with low stock in Electronics'")
    print("   • Reviews: 'Products with highest ratings'")
    print("\nType 'quit' to exit")

    while True:
        try:
            user_question = input("\n💬 Your complex question: ").strip()
            if user_question.lower() in ['quit', 'exit', 'q']:
                break
            if user_question:
                query_complex_database(vn, user_question)
        except KeyboardInterrupt:
            break

    print("\n👋 Thanks for exploring the Complex Natural Language to SQL system!")
    print("🎉 You now have a powerful tool for analyzing multi-table data!")

# Additional utility functions for complex database

def analyze_database_structure():
    """Analyze and display database structure"""
    conn = sqlite3.connect('ecommerce_complex.db')
    cursor = conn.cursor()

    print("\n📋 Database Structure Analysis:")
    print("=" * 50)

    # Get all tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    for table in tables:
        table_name = table[0]
        print(f"\n🔸 Table: {table_name}")

        # Get column info
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = cursor.fetchall()

        for col in columns:
            print(f"   • {col[1]} ({col[2]})")

        # Get row count
        cursor.execute(f"SELECT COUNT(*) FROM {table_name};")
        count = cursor.fetchone()[0]
        print(f"   📊 Records: {count}")

    conn.close()

def suggest_questions():
    """Suggest sample questions for different complexity levels"""

    suggestions = {
        "🔰 Beginner (Single Table)": [
            "How many customers do we have?",
            "What is the most expensive product?",
            "List all product categories",
            "Show customers from California"
        ],

        "🔸 Intermediate (Simple Joins)": [
            "Show products with their categories",
            "List customers with their addresses",
            "Show orders with customer names",
            "Display products with supplier information"
        ],

        "🔹 Advanced (Complex Joins & Aggregation)": [
            "Which customers have the highest lifetime value?",
            "Show monthly sales trends by category",
            "Analyze supplier performance metrics",
            "Calculate average ratings by product category"
        ],

        "🔸 Expert (Multi-table Analytics)": [
            "Show customer purchase patterns across categories",
            "Analyze inventory turnover by supplier",
            "Compare sales performance by geographic region",
            "Calculate customer acquisition and retention metrics"
        ]
    }

    print("\n💡 Suggested Questions by Complexity Level:")
    print("=" * 60)

    for level, questions in suggestions.items():
        print(f"\n{level}:")
        for i, question in enumerate(questions, 1):
            print(f"   {i}. {question}")

if __name__ == "__main__":
    # Run the complex demo
    main_complex()

🚀 Starting Complex Multi-Table Natural Language to SQL Demo

📦 Creating complex e-commerce database...
✅ Complex database schema created successfully!
✅ Complex database populated with realistic data!

⚙️  Setting up Vanna with advanced configuration...


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionAddEvent: capture() takes 1 positional argument but 3 were given



📚 Training on complex database schema...
Adding ddl: CREATE TABLE customers (
            customer_id INTEGER PRIMARY KEY,
            first_name TEXT NOT NULL,
            last_name TEXT NOT NULL,
            email TEXT UNIQUE NOT NULL,
            phone TEXT,
            date_of_birth DATE,
            registration_date DATETIME,
            customer_status TEXT DEFAULT 'active',
            address_id INTEGER,
            FOREIGN KEY (address_id) REFERENCES addresses(address_id)
        );
Adding ddl: CREATE TABLE addresses (
            address_id INTEGER PRIMARY KEY,
            street_address TEXT NOT NULL,
            city TEXT NOT NULL,
            state TEXT NOT NULL,
            zip_code TEXT NOT NULL,
            country TEXT DEFAULT 'USA'
        );
Adding ddl: CREATE TABLE categories (
            category_id INTEGER PRIMARY KEY,
            category_name TEXT NOT NULL,
            parent_category_id INTEGER,
            description TEXT,
            FOREIGN KEY (parent_c

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionAddEvent: capture() takes 1 positional argument but 3 were given


✅ Complex schema training completed!

📝 Training on advanced query examples...


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


✅ Complex query training completed!

🧪 Testing complex multi-table queries...

🤔 Question: How many customers do we have and what is our total revenue?


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


SQL Prompt: [{'role': 'system', 'content': "You are a SQLite expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \nCREATE TABLE customers (\n            customer_id INTEGER PRIMARY KEY,\n            first_name TEXT NOT NULL,\n            last_name TEXT NOT NULL,\n            email TEXT UNIQUE NOT NULL,\n            phone TEXT,\n            date_of_birth DATE,\n            registration_date DATETIME,\n            customer_status TEXT DEFAULT 'active',\n            address_id INTEGER,\n            FOREIGN KEY (address_id) REFERENCES addresses(address_id)\n        );\n\nCREATE TABLE orders (\n            order_id INTEGER PRIMARY KEY,\n            customer_id INTEGER NOT NULL,\n            order_date DATETIME NOT NULL,\n            required_date DATE,\n            shipped_date DATE,\n            ship_address_id INTEGER,\n            freight REAL DE

In [ ]:
#- "Show customer lifetime value with purchase patterns"
#- "Which products in Electronics category have ratings above 4?"
#- "Compare supplier performance by total sales generated"
#- "Show monthly sales trends broken down by payment method"
#- "List customers who haven't ordered in the last 30 days"
#- "Calculate inventory turnover rate by category"
#- "Show geographic distribution of high-value customers"
#- "Analyze review sentiment for products under $100"

# Not using vanna.ai

## Step-by-Step Architecture:

#### Step 1: Core Components

- Database Schema Parser - Extract table structures and relationships
- Query Embeddings - Convert natural language to vector representations
- Training Data Manager - Store example question-SQL pairs
- SQL Generator - Use LLM to generate SQL from context
- Query Executor - Run SQL and return formatted results
- Similarity Matcher - Find relevant examples for context

#### Step 2: Technology Stack

- Database: SQLite (easy to start with)
- Embeddings: OpenAI text-embedding-ada-002
- Vector Storage: Simple in-memory or file-based
- LLM: OpenAI GPT-3.5/GPT-4 for SQL generation
- Vector Similarity: Cosine similarity for matching

#### Step 3: Data Flow

- Parse database schema → Extract table/column info
- Store training examples → Convert to embeddings
- User asks question → Convert to embedding
- Find similar examples → Retrieve relevant context
- Generate SQL → Send context + question to LLM
- Execute & format → Return results to user


In [19]:
# Custom Natural Language to SQL System
# Built from scratch without Vanna.ai - Full control implementation

import os
import sqlite3
import pandas as pd
import numpy as np
import json
import openai
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import warnings
warnings.filterwarnings('ignore')

# Install required packages:
# pip install openai pandas numpy scikit-learn

@dataclass
class TrainingExample:
    """Structure for storing training examples"""
    question: str
    sql: str
    embedding: Optional[np.ndarray] = None
    description: Optional[str] = None

@dataclass
class DatabaseSchema:
    """Structure for database schema information"""
    tables: Dict[str, Dict]
    relationships: List[Dict]
    sample_data: Dict[str, pd.DataFrame]

class CustomNLToSQL:
    """Custom Natural Language to SQL system"""

    def __init__(self, openai_api_key: str, db_path: str):
        """Initialize the NL to SQL system"""
        self.openai_api_key = openai_api_key
        self.db_path = db_path
        self.client = openai.OpenAI(api_key=openai_api_key)

        # Storage for training data and embeddings
        self.training_examples: List[TrainingExample] = []
        self.schema: Optional[DatabaseSchema] = None

        # Configuration
        self.embedding_model = "text-embedding-ada-002"
        self.llm_model = "gpt-4o-mini"
        self.max_examples = 5  # Number of similar examples to include in context

    # It connects to the dtaabase
    # It discovers all the tables automatically
    # For each table it extracts , column names and data types
    # it also extracts first 3 rows
    # It also finds out relationship between the tables
    # I am not giving it create table explicitly
    def connect_and_analyze_database(self):
        """Connect to database and analyze its structure"""
        print("🔍 Analyzing database structure...")

        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        # Get all tables
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        table_names = [row[0] for row in cursor.fetchall()]

        tables = {}
        sample_data = {}

        for table_name in table_names:
            # Get column information
            cursor.execute(f"PRAGMA table_info({table_name});")
            columns = cursor.fetchall()

            # Get sample data (first 3 rows)
            cursor.execute(f"SELECT * FROM {table_name} LIMIT 3;")
            sample_rows = cursor.fetchall()
            column_names = [col[1] for col in columns]

            tables[table_name] = {
                'columns': columns,
                'column_names': column_names,
                'sample_data': sample_rows
            }

            # Store as DataFrame for easier access
            sample_data[table_name] = pd.DataFrame(sample_rows, columns=column_names)

        # Detect relationships (simple foreign key detection)
        relationships = self._detect_relationships(tables)

        self.schema = DatabaseSchema(
            tables=tables,
            relationships=relationships,
            sample_data=sample_data
        )

        conn.close()
        print(f"✅ Analyzed {len(table_names)} tables: {', '.join(table_names)}")

    # Automatically detects foreign key relationship
    # customer_id is orders tables ---> id in customer table
    def _detect_relationships(self, tables: Dict) -> List[Dict]:
        """Simple relationship detection based on column names"""
        relationships = []

        for table1_name, table1_info in tables.items():
            for table2_name, table2_info in tables.items():
                if table1_name != table2_name:
                    # Look for foreign key patterns
                    for col1 in table1_info['column_names']:
                        for col2 in table2_info['column_names']:
                            # Common FK patterns: table2_id in table1, or id in table2
                            if (col1 == f"{table2_name[:-1]}_id" or  # customers -> customer_id
                                (col1.endswith('_id') and col2 == 'id' and
                                 col1.replace('_id', '') in table2_name)):

                                relationships.append({
                                    'from_table': table1_name,
                                    'from_column': col1,
                                    'to_table': table2_name,
                                    'to_column': col2
                                })

        return relationships

    # Converts the users question into vectors
    # uses openai text embedding model

    def get_embeddings(self, texts: List[str]) -> List[np.ndarray]:
        """Get embeddings for a list of texts"""
        try:
            response = self.client.embeddings.create(
                model=self.embedding_model,
                input=texts
            )
            return [np.array(item.embedding) for item in response.data]
        except Exception as e:
            print(f"Error getting embeddings: {e}")
            return [np.zeros(1536) for _ in texts]  # Return zero vectors on error

    def add_training_example(self, question: str, sql: str, description: str = None):
        """Add a training example with its embedding"""
        print(f"📝 Adding training example: {question}")

        # Get embedding for the question
        embeddings = self.get_embeddings([question])

        example = TrainingExample(
            question=question,
            sql=sql,
            embedding=embeddings[0],
            description=description
        )

        self.training_examples.append(example)

    def train_on_examples(self, examples: List[Dict]):
        """Train on a list of question-SQL pairs"""
        print(f"🎓 Training on {len(examples)} examples...")

        questions = [ex['question'] for ex in examples]
        embeddings = self.get_embeddings(questions)

        for i, example in enumerate(examples):
            training_example = TrainingExample(
                question=example['question'],
                sql=example['sql'],
                embedding=embeddings[i],
                description=example.get('description', '')
            )
            self.training_examples.append(training_example)

        print("✅ Training completed!")

    def find_similar_examples(self, question: str, top_k: int = None) -> List[TrainingExample]:
        """Find the most similar training examples to the given question"""
        if not self.training_examples:
            return []

        if top_k is None:
            top_k = self.max_examples

        # Get embedding for the question
        question_embedding = self.get_embeddings([question])[0]

        # Calculate similarities
        similarities = []
        for example in self.training_examples:
            if example.embedding is not None:
                similarity = cosine_similarity(
                    question_embedding.reshape(1, -1),
                    example.embedding.reshape(1, -1)
                )[0][0]
                similarities.append((similarity, example))

        # Sort by similarity and return top k
        similarities.sort(key=lambda x: x[0], reverse=True)
        return [example for _, example in similarities[:top_k]]

    def generate_schema_context(self) -> str:
        """Generate a context string describing the database schema"""
        if not self.schema:
            return ""

        context = "=== DATABASE SCHEMA ===\n\n"

        for table_name, table_info in self.schema.tables.items():
            context += f"Table: {table_name}\n"
            context += "Columns:\n"

            for col in table_info['columns']:
                col_name, col_type = col[1], col[2]
                context += f"  - {col_name} ({col_type})\n"

            # Add sample data
            if not self.schema.sample_data[table_name].empty:
                context += "Sample data:\n"
                sample_df = self.schema.sample_data[table_name]
                context += sample_df.to_string(index=False, max_rows=2)
                context += "\n"

            context += "\n"

        # Add relationships
        if self.schema.relationships:
            context += "=== TABLE RELATIONSHIPS ===\n"
            for rel in self.schema.relationships:
                context += f"{rel['from_table']}.{rel['from_column']} -> {rel['to_table']}.{rel['to_column']}\n"
            context += "\n"

        return context

    def generate_examples_context(self, similar_examples: List[TrainingExample]) -> str:
        """Generate context string from similar examples"""
        if not similar_examples:
            return ""

        context = "=== SIMILAR EXAMPLES ===\n\n"

        for i, example in enumerate(similar_examples, 1):
            context += f"Example {i}:\n"
            context += f"Question: {example.question}\n"
            context += f"SQL: {example.sql}\n\n"

        return context

    def generate_sql(self, question: str) -> str:
        """Generate SQL query from natural language question"""
        print(f"🤔 Processing question: {question}")

        # Find similar examples
        similar_examples = self.find_similar_examples(question)

        # Build context
        schema_context = self.generate_schema_context()
        examples_context = self.generate_examples_context(similar_examples)

        # Create prompt
        prompt = f"""You are an expert SQL query generator. Generate a SQL query to answer the given question.

{schema_context}

{examples_context}

=== INSTRUCTIONS ===
1. Generate only valid SQL that works with the provided schema
2. Use appropriate JOINs when querying multiple tables
3. Include proper WHERE clauses and aggregations as needed
4. Return only the SQL query, no explanations
5. Use SQLite syntax

=== QUESTION ===
{question}

=== SQL QUERY ==="""

        try:
            response = self.client.chat.completions.create(
                model=self.llm_model,
                messages=[
                    {"role": "system", "content": "You are an expert SQL generator. Return only SQL queries."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.1,
                max_tokens=500
            )

            sql_query = response.choices[0].message.content.strip()

            # Clean up the SQL (remove code blocks if present)
            if sql_query.startswith('```'):
                lines = sql_query.split('\n')
                sql_query = '\n'.join(lines[1:-1]) if len(lines) > 2 else sql_query

            print(f"🔍 Generated SQL: {sql_query}")
            return sql_query

        except Exception as e:
            print(f"❌ Error generating SQL: {e}")
            return None

    def execute_sql(self, sql: str) -> pd.DataFrame:
        """Execute SQL query and return results as DataFrame"""
        try:
            conn = sqlite3.connect(self.db_path)
            result = pd.read_sql_query(sql, conn)
            conn.close()
            return result
        except Exception as e:
            print(f"❌ Error executing SQL: {e}")
            return pd.DataFrame()

    def ask_question(self, question: str) -> Tuple[Optional[str], pd.DataFrame]:
        """Main method: ask a question and get SQL + results"""
        print(f"\n🎯 Question: {question}")
        print("=" * 60)

        # Generate SQL
        sql = self.generate_sql(question)
        if not sql:
            return None, pd.DataFrame()

        # Execute SQL
        results = self.execute_sql(sql)

        # Display results
        print(f"\n📊 Results ({len(results)} rows):")
        if results.empty:
            print("No results found")
        else:
            print(results.to_string(index=False))

        return sql, results

    def save_training_data(self, filepath: str):
        """Save training data to file"""
        with open(filepath, 'wb') as f:
            pickle.dump(self.training_examples, f)
        print(f"💾 Training data saved to {filepath}")

    def load_training_data(self, filepath: str):
        """Load training data from file"""
        try:
            with open(filepath, 'rb') as f:
                self.training_examples = pickle.load(f)
            print(f"📂 Training data loaded from {filepath}")
        except FileNotFoundError:
            print(f"⚠️  Training data file not found: {filepath}")

def create_sample_database():
    """Create a sample database with customers and orders"""
    print("🏗️  Creating sample database...")

    conn = sqlite3.connect('sample_ecommerce.db')
    cursor = conn.cursor()

    # Create customers table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS customers (
            customer_id INTEGER PRIMARY KEY,
            first_name TEXT NOT NULL,
            last_name TEXT NOT NULL,
            email TEXT UNIQUE NOT NULL,
            city TEXT,
            state TEXT,
            registration_date DATE
        )
    ''')

    # Create orders table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS orders (
            order_id INTEGER PRIMARY KEY,
            customer_id INTEGER,
            order_date DATE NOT NULL,
            total_amount DECIMAL(10,2),
            status TEXT DEFAULT 'pending',
            FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
        )
    ''')

    # Insert sample customers
    customers_data = [
        (1, 'John', 'Doe', 'john.doe@email.com', 'New York', 'NY', '2023-01-15'),
        (2, 'Jane', 'Smith', 'jane.smith@email.com', 'Los Angeles', 'CA', '2023-02-20'),
        (3, 'Bob', 'Johnson', 'bob.johnson@email.com', 'Chicago', 'IL', '2023-03-10'),
        (4, 'Alice', 'Brown', 'alice.brown@email.com', 'Houston', 'TX', '2023-04-05'),
        (5, 'Charlie', 'Wilson', 'charlie.wilson@email.com', 'Phoenix', 'AZ', '2023-05-18'),
        (6, 'Diana', 'Davis', 'diana.davis@email.com', 'Philadelphia', 'PA', '2023-06-25'),
        (7, 'Edward', 'Miller', 'edward.miller@email.com', 'San Antonio', 'TX', '2023-07-12'),
        (8, 'Fiona', 'Garcia', 'fiona.garcia@email.com', 'San Diego', 'CA', '2023-08-30')
    ]

    cursor.executemany('''
        INSERT OR REPLACE INTO customers
        (customer_id, first_name, last_name, email, city, state, registration_date)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', customers_data)

    # Insert sample orders
    orders_data = [
        (1, 1, '2024-01-15', 1299.99, 'completed'),
        (2, 2, '2024-01-16', 899.50, 'completed'),
        (3, 1, '2024-01-20', 1599.99, 'completed'),
        (4, 3, '2024-01-22', 299.99, 'completed'),
        (5, 4, '2024-01-25', 2199.99, 'completed'),
        (6, 2, '2024-01-28', 449.99, 'completed'),
        (7, 5, '2024-02-01', 799.99, 'completed'),
        (8, 6, '2024-02-05', 1899.99, 'shipped'),
        (9, 3, '2024-02-08', 599.99, 'processing'),
        (10, 7, '2024-02-10', 999.99, 'pending'),
        (11, 8, '2024-02-12', 1299.99, 'completed'),
        (12, 1, '2024-02-15', 399.99, 'completed'),
        (13, 4, '2024-02-18', 1799.99, 'shipped'),
        (14, 5, '2024-02-20', 699.99, 'completed'),
        (15, 2, '2024-02-22', 1099.99, 'processing')
    ]

    cursor.executemany('''
        INSERT OR REPLACE INTO orders
        (order_id, customer_id, order_date, total_amount, status)
        VALUES (?, ?, ?, ?, ?)
    ''', orders_data)

    conn.commit()
    conn.close()
    print("✅ Sample database created successfully!")

def get_training_examples():
    """Define training examples for the system"""
    return [
        {
            "question": "How many customers do we have?",
            "sql": "SELECT COUNT(*) as customer_count FROM customers;",
            "description": "Count total customers"
        },
        {
            "question": "What is our total revenue?",
            "sql": "SELECT SUM(total_amount) as total_revenue FROM orders WHERE status = 'completed';",
            "description": "Sum of completed order amounts"
        },
        {
            "question": "Show all customers from California",
            "sql": "SELECT * FROM customers WHERE state = 'CA';",
            "description": "Filter customers by state"
        },
        {
            "question": "List all orders with customer names",
            "sql": """SELECT o.order_id, c.first_name, c.last_name, o.order_date, o.total_amount, o.status
                     FROM orders o
                     JOIN customers c ON o.customer_id = c.customer_id;""",
            "description": "Join orders with customer information"
        },
        {
            "question": "Which customers have spent the most money?",
            "sql": """SELECT c.first_name, c.last_name, SUM(o.total_amount) as total_spent
                     FROM customers c
                     JOIN orders o ON c.customer_id = o.customer_id
                     WHERE o.status = 'completed'
                     GROUP BY c.customer_id, c.first_name, c.last_name
                     ORDER BY total_spent DESC;""",
            "description": "Customer spending analysis with grouping"
        },
        {
            "question": "How many orders are pending?",
            "sql": "SELECT COUNT(*) as pending_orders FROM orders WHERE status = 'pending';",
            "description": "Count orders by status"
        },
        {
            "question": "Show customers who have made more than 2 orders",
            "sql": """SELECT c.first_name, c.last_name, COUNT(o.order_id) as order_count
                     FROM customers c
                     JOIN orders o ON c.customer_id = o.customer_id
                     GROUP BY c.customer_id, c.first_name, c.last_name
                     HAVING COUNT(o.order_id) > 2;""",
            "description": "Customers with multiple orders using HAVING"
        },
        {
            "question": "What is the average order amount?",
            "sql": "SELECT AVG(total_amount) as avg_order_amount FROM orders;",
            "description": "Calculate average order value"
        },
        {
            "question": "Show orders from February 2024",
            "sql": "SELECT * FROM orders WHERE order_date LIKE '2024-02%';",
            "description": "Filter orders by date range"
        },
        {
            "question": "List customers with their total order count and spending",
            "sql": """SELECT c.first_name, c.last_name, c.city, c.state,
                     COUNT(o.order_id) as total_orders,
                     COALESCE(SUM(o.total_amount), 0) as total_spent
                     FROM customers c
                     LEFT JOIN orders o ON c.customer_id = o.customer_id
                     GROUP BY c.customer_id, c.first_name, c.last_name, c.city, c.state
                     ORDER BY total_spent DESC;""",
            "description": "Complete customer analysis with LEFT JOIN"
        }
    ]

def main():
    """Main function to demonstrate the custom NL to SQL system"""
    print(" Custom Natural Language to SQL System")
    print("=" * 50)

    # IMPORTANT: Set your OpenAI API key here
    openai_api_key = "sk-"

    #if openai_api_key == "your-openai-api-key-here":
    #    print(" Please set your OpenAI API key in the code!")
    #    return

    # Step 1: Create sample database
    create_sample_database()

    # Step 2: Initialize the NL to SQL system
    print("\n  Initializing NL to SQL system...")
    nl_sql = CustomNLToSQL(openai_api_key, 'sample_ecommerce.db')

    # Step 3: Analyze database structure
    nl_sql.connect_and_analyze_database()

    # Step 4: Train on examples
    training_examples = get_training_examples()
    nl_sql.train_on_examples(training_examples)

    # Step 5: Test with sample questions
    print("\n Testing with sample questions...")

    test_questions = [
        "How many customers are there?",
        "What's our total revenue from completed orders?",
        "Show me customers from Texas",
        "Which customer has spent the most money?",
        "List all pending orders with customer details",
        "What's the average order value?",
        "Show customers who registered in 2023",
        "How many orders were placed in February?"
    ]

    for question in test_questions:
        nl_sql.ask_question(question)
        print("\n" + "-" * 60)

    # Step 6: Interactive mode
    print("\n Interactive Mode - Ask your own questions!")
    print("Type 'quit' to exit")

    while True:
        try:
            user_question = input("\n💬 Your question: ").strip()
            if user_question.lower() in ['quit', 'exit', 'q']:
                break
            if user_question:
                nl_sql.ask_question(user_question)
        except KeyboardInterrupt:
            break

    print("\n Thank you for using the Custom NL to SQL system!")

if __name__ == "__main__":
    main()

 Custom Natural Language to SQL System
🏗️  Creating sample database...
✅ Sample database created successfully!

  Initializing NL to SQL system...
🔍 Analyzing database structure...
✅ Analyzed 2 tables: customers, orders
🎓 Training on 10 examples...
✅ Training completed!

 Testing with sample questions...

🎯 Question: How many customers are there?
🤔 Processing question: How many customers are there?
🔍 Generated SQL: SELECT COUNT(*) as customer_count FROM customers;

📊 Results (1 rows):
 customer_count
              8

------------------------------------------------------------

🎯 Question: What's our total revenue from completed orders?
🤔 Processing question: What's our total revenue from completed orders?
🔍 Generated SQL: SELECT SUM(total_amount) as total_revenue FROM orders WHERE status = 'completed';

📊 Results (1 rows):
 total_revenue
       9949.41

------------------------------------------------------------

🎯 Question: Show me customers from Texas
🤔 Processing question: Show me

In [ ]:
# Text to SQL ideally concepts and workflow
# Input : People will write generic Natural language queries.
     #Who are top earners in company X ?
     # Which geo we have seen degrowth last quarter
# Generate SQL :
# It will hit the DB - get the final data

# Steps of developing Text to SQL without using any library from scratch :

- We will have to create a RAG ( Database Schema + Relationship between tables + Training examples ( Natural language query and SQL results )

- We will store all this info in Vector DB after that of Embedding

- When user asks a question it will get coverted to embedding

- It will pull out the relevant tables , sample queries etc from teh vector db corresponding to the question

- It will pass user question + everything else it found to the LLM

- LLM generates a SQL for it

- using that SQL query the agent connects with SQL Db and gets the result
